<a href="https://colab.research.google.com/github/sovandas3/AI-CODE/blob/main/AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# 0.1: Enable GPU in Colab: Runtime -> Change runtime type -> GPU
# 0.2: Mount Google Drive so persistence survives sessions
from google.colab import drive
drive.mount('/content/drive/')
# Choose a folder to store DB files e.g. /content/drive/MyDrive/rag_chroma

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:


# 2) Set path to your folder (change to the shortcut path you created)



FOLDER_PATH = "/content/drive/MyDrive/Dataset/Kiran_4+years_DataAnalyst_SQL_PowerBI_Excel.pdf"   # <- update this to the actual folder name
#FOLDER_PATH = "/content/drive/MyDrive/Dataset"   # <- update this to the actual folder name

In [ ]:
import os

print("Files in folder:", os.listdir(FOLDER_PATH))

NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Dataset/Kiran_4+years_DataAnalyst_SQL_PowerBI_Excel.pdf'

In [ ]:
#!pip install -q chromadb sentence-transformers transformers accelerate faiss-cpu pdfplumber python-docx nltk

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE

In [ ]:
# Persistence directory for DB (in Drive so it survives sessions)
PERSIST_DIR = "/content/drive/MyDrive/rag_chroma_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

In [ ]:
# Create PersistentClient
client = chromadb.PersistentClient(
    path=PERSIST_DIR,
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

In [ ]:
# Then create / get a collection the same way
collection = client.get_or_create_collection(name="docs_collection")
print("✅ Chroma persistent client ready. Path:", PERSIST_DIR)

✅ Chroma persistent client ready. Path: /content/drive/MyDrive/rag_chroma_db


In [ ]:
from pathlib import Path
import pdfplumber, docx
from nltk.tokenize import sent_tokenize
import math

In [ ]:

import nltk

# download standard models (punkt) and a few helpful resources
nltk.download('punkt')
nltk.download('punkt_tab')   # try this if package expects punkt_tab (some chroma/versions mention it)
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
print("NLTK resources downloaded")

NLTK resources downloaded


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from nltk.tokenize import sent_tokenize
import math

def chunk_text_safe(text, max_words=300, overlap_words=50):
    # Try using sent_tokenize; if it errors, fallback to simple split
    try:
        sents = sent_tokenize(text)
        if not sents or len(" ".join(sents).strip()) == 0:
            raise ValueError("sent_tokenize returned empty")
    except Exception as e:
        # fallback: naive split into word windows
        words = text.split()
        chunks = []
        step = max_words - overlap_words
        if step <= 0:
            step = max(1, max_words // 2)
        for i in range(0, max(1, len(words)), step):
            chunk_words = words[i:i+max_words]
            if chunk_words:
                chunks.append(" ".join(chunk_words))
        return chunks

    # build chunks from sentences
    chunks, cur, cur_len = [], [], 0
    for s in sents:
        l = len(s.split())
        if cur_len + l > max_words and cur:
            chunks.append(" ".join(cur))
            # keep some overlap using last few sentences
            overlap_keep = max(1, math.floor(overlap_words / 10))
            cur = cur[-overlap_keep:]
            cur_len = sum(len(x.split()) for x in cur)
        cur.append(s)
        cur_len += l
    if cur:
        chunks.append(" ".join(cur))
    return chunks

In [ ]:
def read_txt(path):
    return Path(path).read_text(encoding="utf-8", errors="ignore")

def read_docx(path):
    doc = docx.Document(path)
    return "\n".join([p.text for p in doc.paragraphs])

def read_pdf(path):
    text = []
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                text.append(t)
    return "\n".join(text)

def extract_text(path):
    p = Path(path)
    suffix = p.suffix.lower()
    if suffix in [".txt", ".md"]:
        return read_txt(p)
    if suffix == ".pdf":
        return read_pdf(p)
    if suffix == ".docx":
        return read_docx(p)
    # fallback - try reading as text
    return read_txt(p)

def chunk_text(text, max_words=300, overlap_words=50):
    sents = sent_tokenize(text)
    chunks, cur, cur_len = [], [], 0
    for s in sents:
        l = len(s.split())
        if cur_len + l > max_words and cur:
            chunks.append(" ".join(cur))
            # naive overlap: keep last few sentences for context
            cur = cur[-3:]
            cur_len = sum(len(x.split()) for x in cur)
        cur.append(s); cur_len += l
    if cur:
        chunks.append(" ".join(cur))
    return chunks

In [ ]:
# Embedding model (local, free)
EMB_MODEL = "all-MiniLM-L6-v2"  # sentence-transformers model
embed_model = SentenceTransformer(EMB_MODEL)

In [ ]:
# Replace with your specific file path
test_file = "/content/drive/MyDrive/Dataset/Kiran_4+years_DataAnalyst_SQL_PowerBI_Excel.pdf"

# use ingest function but call chunk_text_safe
def ingest_file_to_chroma_safe(file_path):
    try:
        text = extract_text(file_path)
    except Exception as e:
        print("Failed to extract text:", e)
        return 0

    if not text or len(text.strip()) == 0:
        print("No text extracted from:", file_path)
        return 0

    try:
        chunks = chunk_text_safe(text, max_words=300, overlap_words=50)
    except Exception as e:
        print("Chunking failed:", e)
        return 0

    if not chunks:
        print("No chunks produced for", file_path)
        return 0

    try:
        embeddings = embed_model.encode(chunks, show_progress_bar=True).tolist()
        ids = [f"{Path(file_path).stem}_c{i}" for i in range(len(chunks))]
        metadatas = [{"source": str(file_path), "chunk_index": i} for i in range(len(chunks))]
        collection.add(ids=ids, documents=chunks, metadatas=metadatas, embeddings=embeddings)
        client.persist()
        print(f"Ingested {file_path} -> {len(chunks)} chunks")
        return len(chunks)
    except Exception as e:
        print("Failed to ingest", file_path, ":\n", e)
        return 0

# run test ingest
ingested = ingest_file_to_chroma_safe(test_file)
print("Ingester returned:", ingested)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to ingest /content/drive/MyDrive/Dataset/Kiran_4+years_DataAnalyst_SQL_PowerBI_Excel.pdf :
 'Client' object has no attribute 'persist'
Ingester returned: 0


In [ ]:

def ingest_file_to_chroma(file_path):
    text = extract_text(file_path)
    if not text or len(text.strip()) == 0:
        print("No text extracted from:", file_path)
        return 0
    chunks = chunk_text(text, max_words=300, overlap_words=50)
    embeddings = embed_model.encode(chunks, show_progress_bar=False).tolist()
    ids = [f"{Path(file_path).stem}_c{i}" for i in range(len(chunks))]
    metadatas = [{"source": str(file_path), "chunk_index": i} for i in range(len(chunks))]
    collection.add(ids=ids, documents=chunks, metadatas=metadatas, embeddings=embeddings)
    client.persist()
    print(f"Ingested {file_path} -> {len(chunks)} chunks")
    return len(chunks)

In [ ]:
from pathlib import Path
DATA_FOLDER = FOLDER_PATH  # from Cell 1

In [ ]:

p = Path(FOLDER_PATH)
if not p.exists():
    raise SystemExit("DATA_FOLDER not found - update DRIVE_FOLDER path (Cell 1).")

total_chunks = 0
for f in sorted(p.glob("*")):
    if f.suffix.lower() in [".pdf", ".txt", ".md", ".docx"]:
        try:
            total_chunks += ingest_file_to_chroma(str(f))
        except Exception as e:
            print("Failed to ingest", f, ":", e)

print("Bulk ingest finished. Total chunks:", total_chunks)

Bulk ingest finished. Total chunks: 0


In [ ]:
def retrieve(query, top_k=5):
    q_emb = embed_model.encode([query]).tolist()
    results = collection.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas","distances"])
    rows = results['results'][0]
    docs = [{"text": t, "meta": m, "score": s} for t,m,s in zip(rows['documents'], rows['metadatas'], rows['distances'])]
    return docs

In [ ]:
# Quick sanity check (after ingestion)
if collection.count() > 0:
    print("Collection size (items):", collection.count())
else:
    print("Collection appears empty. Did ingestion run correctly?")

Collection size (items): 3


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

GEN_MODEL = "google/flan-t5-base"  # choose smaller if GPU RAM is limited

In [ ]:
device = 0 if torch.cuda.is_available() else -1

print("Loading generator model (this may take a minute)...")
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL).to("cuda" if torch.cuda.is_available() else "cpu")
gen_pipeline = pipeline("text2text-generation", model=gen_model, tokenizer=tokenizer, device=device)
print("Generator ready on device:", "cuda" if device==0 else "cpu")

Loading generator model (this may take a minute)...


Device set to use cpu


Generator ready on device: cpu


In [ ]:
def assemble_prompt(query, docs):
    prompt = "Answer the question using ONLY the following sources. If the answer is not present, say 'I don't know'.\n\n"
    prompt += f"User question: {query}\n\n"
    for i, d in enumerate(docs):
        m = d["meta"]
        prompt += f"[S{i}] {m.get('source')} (chunk {m.get('chunk_index')})\n{d['text']}\n\n"
    prompt += "\nAnswer concisely and include source citations like [S0], [S1].\n"
    return prompt

def rag_answer(query, top_k=5, max_new_tokens=200):
    docs = retrieve(query, top_k=top_k)
    prompt = assemble_prompt(query, docs)
    out = gen_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    return {"answer": out[0]['generated_text'], "sources": docs}

In [ ]:
# Run this to print the exact structure Chroma returns for a query (no assumptions).
query = "What is a vector database in simple terms?"
q_emb = embed_model.encode([query]).tolist()

try:
    raw = collection.query(query_embeddings=q_emb, n_results=4, include=["documents","metadatas","distances"])
    import pprint, json
    print("===== raw type:", type(raw))
    pprint.pprint(raw)
except Exception as e:
    print("collection.query raised:", repr(e))

===== raw type: <class 'dict'>
{'data': None,
 'distances': [[1.5077815055847168, 1.570183277130127, 1.5844018459320068]],
 'documents': [['KIRAN KUMAR LENKA\n'
                'Data Analyst\n'
                '● Hyderabad, India 751024 ● 9178052229 ● '
                'kirankumar1.analyst@gmail.com\n'
                '● https://www.linkedin.com/in/kiran-kumar-lenka\n'
                'PROFESSIONAL SUMMARY\n'
                'Detail-oriented and results-driven Data Analyst with 4+ years '
                'of experience in collecting, analyzing,\n'
                'and interpreting complex data to drive business decisions. '
                'Skilled in SQL, Python, Excel, Power BI,\n'
                'and Tableau with proven expertise in building dashboards, '
                'generating actionable insights, and\n'
                'optimizing reporting processes. Strong background in data '
                'visualization, statistical analysis, and\n'
                'business intelligen

In [ ]:
# Replace previous retrieve() with this robust version
def retrieve_safe(query, top_k=5, debug=False):
    q_emb = embed_model.encode([query]).tolist()
    try:
        results = collection.query(query_embeddings=q_emb, n_results=top_k, include=["documents","metadatas","distances"])
    except Exception as e:
        if debug:
            print("Chroma query exception:", e)
        return []

    # If the result already has 'results' as expected:
    if isinstance(results, dict) and 'results' in results and isinstance(results['results'], list) and len(results['results'])>0:
        rows = results['results'][0]
    # Some versions return a list directly
    elif isinstance(results, list) and len(results)>0 and isinstance(results[0], dict):
        rows = results[0]
    # Some versions return a dict with a single key pointing to the payload
    else:
        # Debug print of unexpected shape
        if debug:
            import pprint
            print("Unexpected query result shape:", type(results))
            pprint.pprint(results)
        return []

    # Now extract documents/metadatas/distances robustly
    docs_list = rows.get('documents') or rows.get('document') or rows.get('documents_list') or []
    metas_list = rows.get('metadatas') or rows.get('metas') or rows.get('metadata') or []
    dists_list = rows.get('distances') or rows.get('scores') or rows.get('distance') or []

    # If any list is empty but others are present, try to align lengths
    max_len = max(len(docs_list), len(metas_list), len(dists_list))
    docs = []
    for i in range(max_len):
        txt = docs_list[i] if i < len(docs_list) else ""
        meta = metas_list[i] if i < len(metas_list) else {}
        score = dists_list[i] if i < len(dists_list) else None
        if txt or meta:
            docs.append({"text": txt, "meta": meta, "score": score})

    return docs

In [ ]:
def rag_answer_safe(query, top_k=5, max_new_tokens=200, debug=False):
    docs = retrieve_safe(query, top_k=top_k, debug=debug)
    if not docs:
        if debug:
            print("No docs retrieved. Returning fallback answer.")
        return {"answer": "No relevant sources found.", "sources": []}
    prompt = assemble_prompt(query, docs)
    out = gen_pipeline(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    return {"answer": out[0]['generated_text'], "sources": docs}

In [ ]:

# Test it
q = "What is a vector database in simple terms?"
resp = rag_answer_safe(q, top_k=4, debug=True)
print("ANSWER:\n", resp["answer"])
print("\nSOURCES (meta):")
for s in resp["sources"]:
    print(s["meta"])

Unexpected query result shape: <class 'dict'>
{'data': None,
 'distances': [[1.5077815055847168, 1.570183277130127, 1.5844018459320068]],
 'documents': [['KIRAN KUMAR LENKA\n'
                'Data Analyst\n'
                '● Hyderabad, India 751024 ● 9178052229 ● '
                'kirankumar1.analyst@gmail.com\n'
                '● https://www.linkedin.com/in/kiran-kumar-lenka\n'
                'PROFESSIONAL SUMMARY\n'
                'Detail-oriented and results-driven Data Analyst with 4+ years '
                'of experience in collecting, analyzing,\n'
                'and interpreting complex data to drive business decisions. '
                'Skilled in SQL, Python, Excel, Power BI,\n'
                'and Tableau with proven expertise in building dashboards, '
                'generating actionable insights, and\n'
                'optimizing reporting processes. Strong background in data '
                'visualization, statistical analysis, and\n'
                'busi

In [ ]:

# Test it
q = "Summarize the pdf in short and simple"
resp = rag_answer_safe(q, top_k=4, debug=True)
print("ANSWER:\n", resp["answer"])
print("\nSOURCES (meta):")
for s in resp["sources"]:
    print(s["meta"])

Unexpected query result shape: <class 'dict'>
{'data': None,
 'distances': [[1.8508903980255127, 1.8653703927993774, 1.9691920280456543]],
 'documents': [['Key Responsibilities:\n'
                '• Gathered data from business and accounts teams, focusing on '
                'loyalty marketing needs. • Used SQL and SSIS for data '
                'extraction, transformation, and loading (ETL) to ensure data '
                'quality\n'
                'and readiness. • Utilized Informatica for data cleaning, '
                'ensuring accuracy and reliability. • Developed data models to '
                'support loyalty marketing strategies and managed data '
                'storage\n'
                'efficiently. • Created reports and dashboards in Power BI, '
                'providing insights into loyalty marketing\n'
                'performance. • Updated and customized reports based on client '
                'requests, ensuring timely and relevant data\n'
              

In [ ]:
q = "What is a vector database in simple terms?"
resp = rag_answer(q, top_k=4)
print("ANSWER:\n", resp["answer"])
print("\nSOURCES (meta):")
for s in resp["sources"]:
    print(s["meta"])

KeyError: 'results'